<a href="https://colab.research.google.com/github/Baraa710/Premier_League_Predictor/blob/main/FixturesPlayersScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")


Mounted at /content/drive/


In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import os

In [ ]:
col_names = ["Home","Away","Score"]
i=0
while i<11:
  i+=1
  col_names.append("Home-Player-{}".format(i))
  col_names.append("Home-Player-{}-url".format(i))

i = 0
while i<11:
  i+=1
  col_names.append("Away-Player-{}".format(i))
  col_names.append("Away-Player-{}-url".format(i))




In [ ]:
print(col_names)

['Home', 'Away', 'Score', 'Home-Player-1', 'Home-Player-1-url', 'Home-Player-2', 'Home-Player-2-url', 'Home-Player-3', 'Home-Player-3-url', 'Home-Player-4', 'Home-Player-4-url', 'Home-Player-5', 'Home-Player-5-url', 'Home-Player-6', 'Home-Player-6-url', 'Home-Player-7', 'Home-Player-7-url', 'Home-Player-8', 'Home-Player-8-url', 'Home-Player-9', 'Home-Player-9-url', 'Home-Player-10', 'Home-Player-10-url', 'Home-Player-11', 'Home-Player-11-url', 'Away-Player-1', 'Away-Player-1-url', 'Away-Player-2', 'Away-Player-2-url', 'Away-Player-3', 'Away-Player-3-url', 'Away-Player-4', 'Away-Player-4-url', 'Away-Player-5', 'Away-Player-5-url', 'Away-Player-6', 'Away-Player-6-url', 'Away-Player-7', 'Away-Player-7-url', 'Away-Player-8', 'Away-Player-8-url', 'Away-Player-9', 'Away-Player-9-url', 'Away-Player-10', 'Away-Player-10-url', 'Away-Player-11', 'Away-Player-11-url']


In [ ]:
seasons = ["{}-{}".format(n,n+1) for n in range(2005,2020)]
seasons

['2005-2006',
 '2006-2007',
 '2007-2008',
 '2008-2009',
 '2009-2010',
 '2010-2011',
 '2011-2012',
 '2012-2013',
 '2013-2014',
 '2014-2015',
 '2015-2016',
 '2016-2017',
 '2017-2018',
 '2018-2019',
 '2019-2020']

In [ ]:
def scrape_fixtures_with_players(season):
  # Find the table with the specified ID
  url ='https://fbref.com/en/comps/9/{}/schedule/{}-Premier-League-Scores-and-Fixtures'.format(season,season)
  html_response = requests.get(url)

  while html_response.status_code!=200:
    print(html_response.status_code)
    time.sleep(1200)
    html_response = requests.get(url)

  soup = BeautifulSoup(html_response.content, "html.parser")
  table = soup.find('table', {'class':'stats_table'})
  # Extract all rows from the table
  rows = table.find_all('tr')

  # iterate the rows
  count=0
  allRows = []
  for i, row in enumerate(rows):
      currentRow = []
      if(i==0):
        continue

      # Get the href for the match report
      match_report_cell = row.find('td', {'data-stat': 'match_report'})
      if(match_report_cell.text.strip()=='Head-to-Head'):
        continue
      home_team = row.find('td',{'data-stat':'home_team'}).text.strip()
      away_team = row.find('td', {'data-stat':'away_team'}).text.strip()
      score = row.find('td', {'data-stat':'score'}).text.strip()
      currentRow.append(home_team)
      currentRow.append(away_team)
      currentRow.append(score)

      if match_report_cell:
        match_report_link = match_report_cell.find('a')
        if match_report_link:
          count+=1
          match_report_url = "https://fbref.com"+match_report_link['href']
          response = requests.get(match_report_url)
          time.sleep(1.5)
          while response.status_code!=200:
            print(response.status_code)
            time.sleep(1200)
            response = requests.get(url)
          newSoup = BeautifulSoup(response.content, "html.parser")
          if(count%10==0):
            print(count)

          teamDataHome = newSoup.find("div", {'class': 'lineup', 'id': 'a'})
          squadListHome = teamDataHome.find_all('tr')
          for i, row in enumerate(squadListHome):
            if i>0 and i<12:
              currentRow.append(row.find_all('td')[1].text.strip())
              currentRow.append(row.find('a')['href'])
          teamDataAway = newSoup.find("div", {'class': 'lineup', 'id': 'a'})
          squadListAway = teamDataAway.find_all('tr')
          for i, row in enumerate(squadListAway):
            if i>0 and i<12:
              currentRow.append(row.find_all('td')[1].text.strip())
              currentRow.append(row.find('a')['href'])

      allRows.append(currentRow)
  df = pd.DataFrame(allRows, columns=col_names)
  df = df.replace(r'^s*$', float('NaN'), regex = True)
  df.dropna(inplace = True)
  df.to_excel("/content/drive/MyDrive/APS360_Project/Match_Data/Fixtures_With_Players/{}.xlsx".format(season))
  return allRows


scrape_fixtures_with_players('2023-2024')


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300


[['Burnley',
  'Manchester City',
  '0–3',
  'James Trafford',
  '/en/players/259fea27/James-Trafford',
  "Dara O'Shea",
  '/en/players/1d042188/Dara-OShea',
  'Jordan Beyer',
  '/en/players/9af1f9ff/Jordan-Beyer',
  'Connor Roberts',
  '/en/players/24fae784/Connor-Roberts',
  'Sander Berge',
  '/en/players/d0b6129f/Sander-Berge',
  'Lyle Foster',
  '/en/players/f84a807c/Lyle-Foster',
  'Vitinho',
  '/en/players/9295dc1c/Vitinho',
  'Josh Cullen',
  '/en/players/f6d7d690/Josh-Cullen',
  'Zeki Amdouni',
  '/en/players/2ee5b0c9/Zeki-Amdouni',
  'Ameen Al-Dakhil',
  '/en/players/93e08bce/Ameen-Al-Dakhil',
  'Luca Koleosho',
  '/en/players/fc076f78/Luca-Koleosho',
  'James Trafford',
  '/en/players/259fea27/James-Trafford',
  "Dara O'Shea",
  '/en/players/1d042188/Dara-OShea',
  'Jordan Beyer',
  '/en/players/9af1f9ff/Jordan-Beyer',
  'Connor Roberts',
  '/en/players/24fae784/Connor-Roberts',
  'Sander Berge',
  '/en/players/d0b6129f/Sander-Berge',
  'Lyle Foster',
  '/en/players/f84a807c/

In [ ]:
seasons = ["{}-{}".format(n,n+1) for n in range(2008,2020)]
seasons

['2008-2009',
 '2009-2010',
 '2010-2011',
 '2011-2012',
 '2012-2013',
 '2013-2014',
 '2014-2015',
 '2015-2016',
 '2016-2017',
 '2017-2018',
 '2018-2019',
 '2019-2020']

In [ ]:
import multiprocessing

num_processors = multiprocessing.cpu_count()

pool = multiprocessing.Pool(num_processors)

pool.map(scrape_fixtures_with_players, seasons)


10
10
20
20
30
30
40
40
50
50
60
60
70
70
80
80
90
90
100
110
100
120
110
130
120
130
140
140
150
150
160
160
170
170
180
180
190
190
200
200
210
210
220
220
230
230
240
240
250
250
260
270
260
270
280
280
290
290
300
300
310
310
320
320
330
330
340
340
350
350
360
360
370
370
380
380
10
10
20
20
30
30
40
40
50
50
60
70
60
70
80
90
80
100
90
110
100
120
110
130
120
140
130
140
150
150
160
160
170
180
170
190
180
200
190
210
200
220
210
230
220
230
240
240
250
250
260
260
270
270
280
280
290
300
290
310
300
320
310
330
320
340
330
350
340
360
370
350
360
380
370
10
380
20
10
30
20
40
30
50
40
50
60
60
70
70
80
80
90
90
100
110
100
120
110
130
120
140
150
130
140
160
150
170
180
160
190
170
200
180
210
220
190
230
200
240
210
250
260
220
270
280
230
290
240
300
250
310
260
320
270
330
340
280
290
350
360
300
370
310
380
10
320
403
403
403
403
403
403


Process ForkPoolWorker-36:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Process ForkPoolWorker-35:
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-65-7e98f8fab17b>", line 41, in scrape_fixtures_with_players
    time.sleep(1200)
  File "

KeyboardInterrupt: 